In [1]:
import requests
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import pandas as pd
import psycopg2

In [2]:
def load_secrets():
    load_dotenv()
    env_path = Path(".") / ".env"
    load_dotenv(dotenv_path=env_path)

    API_KEY = os.getenv("API_KEY")
    return API_KEY

In [3]:
API_KEY = load_secrets()

In [4]:
url = f"https://airquality.googleapis.com/v1/currentConditions:lookup?key={API_KEY}"

In [5]:
TIMEZONE_BASE_URL = "https://maps.googleapis.com/maps/api/timezone/json"

In [18]:
TIMEZONE_BASE_URL

'https://maps.googleapis.com/maps/api/timezone/json'

In [7]:
headers = {"Content-Type": "application/json"}

In [8]:
CAPITAIS = {
    "Rio Branco": (-9.97499, -67.8243),
    "Maceió": (-9.66599, -35.735),
    "Macapá": (0.0389, -51.0664),
    "Manaus": (-3.11866, -60.0212),
    "Salvador": (-12.9714, -38.5014),
    "Fortaleza": (-3.73186, -38.5267),
    "Brasília": (-15.7797, -47.9297),
    "Vitória": (-20.3155, -40.3128),
    "Goiânia": (-16.6786, -49.2539),
    "São Luís": (-2.52972, -44.3028),
    "Cuiabá": (-15.5989, -56.0949),
    "Campo Grande": (-20.4428, -54.6464),
    "Belo Horizonte": (-19.9167, -43.9345),
    "Belém": (-1.45502, -48.5024),
    "João Pessoa": (-7.115, -34.8631),
    "Curitiba": (-25.4278, -49.2731),
    "Recife": (-8.05, -34.9),
    "Teresina": (-5.08921, -42.8016),
    "Rio de Janeiro": (-22.9068, -43.1729),
    "Natal": (-5.795, -35.2094),
    "Porto Alegre": (-30.0331, -51.23),
    "Porto Velho": (-8.76194, -63.9039),
    "Boa Vista": (2.81972, -60.6733),
    "Florianópolis": (-27.5969, -48.5495),
    "São Paulo": (-23.5505, -46.6333),
    "Aracaju": (-10.9111, -37.0717),
    "Palmas": (-10.1675, -48.3277)
}

In [9]:
capitais_dict = {}
for capital, coords in CAPITAIS.items():
    response = requests.post(url=url,
                             json={"location": {"latitude": coords[0], "longitude": coords[1]}},
                             headers=headers)
    capitais_dict.update({capital : response.text})

In [10]:
linhas = []
for cidade, data_json in capitais_dict.items():
    cidade_data = json.loads(data_json)
    idx = cidade_data["indexes"][0]
    linhas.append({
        "City": cidade,

            })
# print(linhas)
# cidade_data["indexes"][0].get('aqi')
# cidade_data["indexes"][0]["aqi"]
cidade_data["indexes"][0]["color"]["blue"]

0.2

In [11]:
linhas = []
for cidade, data_json in capitais_dict.items():
    cidade_data = json.loads(data_json)
    idx = cidade_data["indexes"][0]

    linhas.append({
        "City": cidade,
        "Date/Time": cidade_data.get("dateTime"),
        "Local" : cidade_data.get("regionCode"),
        "AQI": idx.get("aqi"),
        "Red": idx.get("color",{}).get("red"),
        "Green": idx.get("color",{}).get("green"),
        "Blue": idx.get("color",{}).get("blue"),
        "Label": idx.get("category"),
        "Dominant Pollutant": idx.get("dominantPollutant")
    })

In [12]:
linhas

[{'City': 'Rio Branco',
  'Date/Time': '2025-09-19T02:00:00Z',
  'Local': 'br',
  'AQI': 72,
  'Red': 0.48235294,
  'Green': 0.79607844,
  'Blue': 0.2,
  'Label': 'Good air quality',
  'Dominant Pollutant': 'pm25'},
 {'City': 'Maceió',
  'Date/Time': '2025-09-19T02:00:00Z',
  'Local': 'br',
  'AQI': 75,
  'Red': 0.43137255,
  'Green': 0.7764706,
  'Blue': 0.20392157,
  'Label': 'Good air quality',
  'Dominant Pollutant': 'o3'},
 {'City': 'Macapá',
  'Date/Time': '2025-09-19T02:00:00Z',
  'Local': 'br',
  'AQI': 77,
  'Red': 0.39607844,
  'Green': 0.7647059,
  'Blue': 0.20392157,
  'Label': 'Good air quality',
  'Dominant Pollutant': 'o3'},
 {'City': 'Manaus',
  'Date/Time': '2025-09-19T02:00:00Z',
  'Local': 'br',
  'AQI': 69,
  'Red': 0.5411765,
  'Green': 0.81960785,
  'Blue': 0.1882353,
  'Label': 'Good air quality',
  'Dominant Pollutant': 'pm25'},
 {'City': 'Salvador',
  'Date/Time': '2025-09-19T02:00:00Z',
  'Local': 'br',
  'AQI': 68,
  'Red': 0.5647059,
  'Green': 0.827451,
  '

In [13]:
df = pd.DataFrame(data=linhas)
df

,City,Date/Time,Local,AQI,Red,Green,Blue,Label,Dominant Pollutant
0,Rio Branco,2025-09-19T02:00:00Z,br,72,0.482353,0.796078,0.200000,Good air quality,pm25
1,Maceió,2025-09-19T02:00:00Z,br,75,0.431373,0.776471,0.203922,Good air quality,o3
2,Macapá,2025-09-19T02:00:00Z,br,77,0.396078,0.764706,0.203922,Good air quality,o3
3,Manaus,2025-09-19T02:00:00Z,br,69,0.541176,0.819608,0.188235,Good air quality,pm25
4,Salvador,2025-09-19T02:00:00Z,br,68,0.564706,0.827451,0.176471,Good air quality,pm10
5,Fortaleza,2025-09-19T02:00:00Z,br,73,0.462745,0.792157,0.200000,Good air quality,o3
6,Brasília,2025-09-19T02:00:00Z,br,39,1.000000,0.749020,NaN,Low air quality,pm10
7,Vitória,2025-09-19T02:00:00Z,br,76,0.411765,0.772549,0.203922,Good air quality,pm25
8,Goiânia,2025-09-19T02:00:00Z,br,54,0.901961,0.960784,0.039216,Moderate air quality,pm25
9,São Luís,2025-09-19T02:00:00Z,br,55,0.878431,0.952941,0.047059,Moderate air quality,pm10


In [16]:
# def preprocess_air_quality():
#     conn = psycopg2.connect(
#         host='Localhost',
#         dbname="meubanco",
#     )sdavadv